In [1]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [2]:
from tensorflow.keras.layers import Input, SimpleRNN, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# N = Number of samples
# T= Sequence length
# D= number of input features
# M= Number of hidden units
# K= number of output units

In [4]:
N=1
T= 10
D=3
K=2
X = np.random.randn(N,T,D)

In [5]:
print(X)

[[[ 1.28145778 -0.42848834  0.94592515]
  [-0.03819839  0.55944254 -0.17545312]
  [ 1.05966203 -0.59520965 -0.04251422]
  [-1.2915637  -0.46141409  0.33353085]
  [ 0.16187552 -0.61826175  0.10057974]
  [ 0.1174313   0.21950943  1.54130488]
  [-0.10380297 -1.37670052  2.1975564 ]
  [ 1.11112099 -0.47640426 -1.71483401]
  [ 0.28147785 -0.83301722  0.08251582]
  [ 0.23851944  1.19388584 -1.00077457]]]


In [6]:
X[0]

array([[ 1.28145778, -0.42848834,  0.94592515],
       [-0.03819839,  0.55944254, -0.17545312],
       [ 1.05966203, -0.59520965, -0.04251422],
       [-1.2915637 , -0.46141409,  0.33353085],
       [ 0.16187552, -0.61826175,  0.10057974],
       [ 0.1174313 ,  0.21950943,  1.54130488],
       [-0.10380297, -1.37670052,  2.1975564 ],
       [ 1.11112099, -0.47640426, -1.71483401],
       [ 0.28147785, -0.83301722,  0.08251582],
       [ 0.23851944,  1.19388584, -1.00077457]])

In [7]:
x = X[0]
for t in range(T):
  print(x[0])
  break

[ 1.28145778 -0.42848834  0.94592515]


In [8]:
# Make an RNN

# Number of hidden units
M=5
# T*D = Input matrix
i = Input(shape= (T,D))
x = SimpleRNN(M)(i)
x = Dense(K)(x)

# No activation for regression
model = Model(i,x)


In [9]:
# Get the output

Yhat = model.predict(X)
print(Yhat)

# output shape = N*K
print(Yhat.shape)

1/1 [==============================] - 1s 861ms/step
[[-1.3271614  -0.02186675]]
(1, 2)


In [10]:
model.summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 3)]           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 45        
                                                                 
 dense (Dense)               (None, 2)                 12        
                                                                 
Total params: 57 (228.00 Byte)
Trainable params: 57 (228.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.layers[0]

In [12]:
model.layers[1]

In [13]:
# See what parameters are stored in RNN layer
model.layers[1].get_weights()

[array([[ 0.82798105, -0.6833321 , -0.58263385, -0.50161743,  0.67022866],
        [ 0.70966417,  0.4056291 , -0.49929702,  0.55681103,  0.6079722 ],
        [-0.62610316, -0.864318  ,  0.5499117 ,  0.8521423 , -0.36800995]],
       dtype=float32),
 array([[-0.17656827, -0.49153155,  0.42900467, -0.02383973,  0.73661846],
        [ 0.02861009,  0.20022757, -0.1290009 , -0.9530562 ,  0.18475132],
        [-0.3205456 , -0.665553  , -0.65243495, -0.0890239 , -0.1438507 ],
        [-0.93006724,  0.33544177,  0.12969738,  0.01059524, -0.07429689],
        [ 0.01518133,  0.4035141 , -0.5973448 ,  0.28823265,  0.63011664]],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32)]

In [14]:
# Check shapes
# First output: Input>Hidden : D*M
# Second output: Hidden > Hidden : M*M
# Third output: Bias Term (vector of length M)

a,b,c = model.layers[1].get_weights()
print(a.shape, b.shape, c.shape)

(3, 5) (5, 5) (5,)


In [15]:
# Output layer weights and bias

model.layers[2].get_weights()

[array([[ 0.2474122 ,  0.18046331],
        [ 0.53704405,  0.07881129],
        [ 0.7603425 , -0.09752744],
        [-0.19696248, -0.41251874],
        [-0.9034281 , -0.22642213]], dtype=float32),
 array([0., 0.], dtype=float32)]

In [16]:
# Wx input to hidden weight
# Wh hidden to hidden weight
# bh hidden bias
# Wo Hidden to output weight
# bo output bias

Wx, Wh, bh = model.layers[1].get_weights()
Wo, bo = model.layers[2].get_weights()

In [17]:
h_last = np.zeros(M)
x = X[0]
Yhats = []

for t in range(T):
  h = np.tanh(x[t].dot(Wx) + h_last.dot(Wh) + bh)
  y = h.dot(Wo) +bo
  Yhats.append(y)

  h_last = h

print(Yhats[-1])

[-1.32716133 -0.02186675]
